In [23]:
from pathlib import Path
from datasets import load_dataset

ProjectPath = Path().resolve().parent

In [51]:
dataset_path = ProjectPath / "data/finetune/processed/"
# finetune_dataset = load_dataset("csv", data_files=str(dataset_path))

In [22]:
len(
    "-- Database schema | artists : id [ INT ] primary_key , name [ TEXT ] | sqlite_sequence : name [ TEXT ] , seq [ TEXT ] | albums : id [ INT ] primary_key , title [ TEXT ] , artist_id [ INT ] albums.artist_id = artists.id | employees : id [ INT ] primary_key , last_name [ TEXT ] , first_name [ TEXT ] , title [ TEXT ] , reports_to [ INT ] employees.reports_to = employees.id , birth_date [ TEXT ] , hire_date [ TEXT ] , address [ TEXT ] , city [ TEXT ] , state [ TEXT ] , country [ TEXT ] , postal_code [ TEXT ] , phone [ TEXT ] , fax [ TEXT ] , email [ TEXT ] | customers : id [ INT ] primary_key , first_name [ TEXT ] , last_name [ TEXT ] , company [ TEXT ] , address [ TEXT ] , city [ TEXT ] , state [ TEXT ] , country [ TEXT ] , postal_code [ TEXT ] , phone [ TEXT ] , fax [ TEXT ] , email [ TEXT ] , support_rep_id [ INT ] customers.support_rep_id = employees.id | genres : id [ INT ] primary_key , name [ TEXT ] | invoices : id [ INT ] primary_key , customer_id [ INT ] invoices.customer_id = customers.id , invoice_date [ TEXT ] , billing_address [ TEXT ] , billing_city [ TEXT ] , billing_state [ TEXT ] , billing_country [ TEXT ] , billing_postal_code [ TEXT ] , total [ INT ] | media_types : id [ INT ] primary_key , name [ TEXT ] | tracks : id [ INT ] primary_key , name [ TEXT ] , album_id [ INT ] tracks.album_id = albums.id , media_type_id [ INT ] tracks.media_type_id = media_types.id , genre_id [ INT ] tracks.genre_id = genres.id , composer [ TEXT ] , milliseconds [ INT ] , bytes [ INT ] , unit_price [ INT ] | invoice_lines : id [ INT ] primary_key , invoice_id [ INT ] invoice_lines.invoice_id = invoices.id , track_id [ INT ] invoice_lines.track_id = tracks.id , unit_price [ INT ] , quantity [ INT ] | playlists : id [ INT ] primary_key , name [ TEXT ] | playlist_tracks : playlist_id [ INT ] primary_key playlist_tracks.playlist_id = playlists.id , track_id [ INT ] playlist_tracks.track_id = tracks.id | -- -- A list of the top 5 countries by number of invoices. List country name and number of invoices."
)

2027

In [46]:
import pandas as pd

df = pd.read_csv(
    dataset_path,
)

In [49]:
# Filter rows where the length of 'input' is less than 3000
df_filter = df[df["input"].str.len() < 3000]

In [57]:
df_filter.to_json(
    dataset_path / "train_filter.jsonl", index=False, orient="records", lines=True
)

In [58]:
import json

system_prompt = "Given the following database schema and natural language question, write the SQL query that answers the question."

with open(dataset_path / "azure_sql_finetune.jsonl", "w") as f_out:
    for _, row in df_filter.iterrows():
        formatted = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": row["input"]},
                {"role": "assistant", "content": row["output"]},
            ]
        }
        f_out.write(json.dumps(formatted) + "\n")